# Preparación y Limpieza de Datos

## Objetivo
Construcción del dataset maestro mediante la integración del historial de consultas y datos de usuarios, aplicando validaciones de calidad y limpieza de datos inconsistentes.

## Fuentes de Datos
- New_HistConsultas.csv: Historial de interacciones en la línea de atención
- New_Usuarios.csv: Características demográficas y comportamentales de clientes

## Proceso
1. Carga y validación inicial
2. Auditoría de calidad de datos
3. Limpieza de registros duplicados e inconsistentes
4. Integración mediante merge
5. Exportación de dataset procesado

In [4]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Rutas de datos
ruta_historial = '../data/raw/New_HistConsultas.csv'
ruta_usuarios = '../data/raw/New_Usuarios.csv'

## 1. Carga de Datos

In [5]:
# Carga con encoding latin-1 para caracteres en español
try:
    df_consultas_raw = pd.read_csv(ruta_historial, encoding='latin-1', sep=';')
    df_usuarios_raw = pd.read_csv(ruta_usuarios, encoding='latin-1', sep=';')
    print(f"Archivos cargados correctamente")
    print(f"Consultas: {df_consultas_raw.shape}")
    print(f"Usuarios: {df_usuarios_raw.shape}")
except Exception as e:
    print(f"Error al cargar archivos: {e}")
    raise

print(f"\nPrimeras filas - Historial de Consultas:")
display(df_consultas_raw.head())

print(f"\nPrimeras filas - Datos de Usuarios:")
display(df_usuarios_raw.head())

Archivos cargados correctamente
Consultas: (1282960, 4)
Usuarios: (337051, 22)

Primeras filas - Historial de Consultas:


,Unnamed: 0,ID_Cuenta,Fecha_consulta,Tipo_consulta
0,0,340,2015-06-22 16:14:35,Actualización datos
1,1,340,2015-06-22 16:37:02,Agendamiento citas
2,2,340,2015-06-22 15:50:56,Agendamiento citas
3,3,4d5c0e16-9bef-8334-687e-55f043e02c62,2015-09-09 14:35:37,Agendamiento citas
4,4,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,2015-07-24 20:36:26,Agendamiento citas



Primeras filas - Datos de Usuarios:


,ID_Cuenta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
0,340,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
1,4d5c0e16-9bef-8334-687e-55f043e02c62,soltero,Santafé de Bogotá,no,f,no,si,si,si,no,mensajero,30167.091798,65,3,m17,124.262540,55.257504,no,no,new-new,si,0
2,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,soltero,Santafé de Bogotá,no,f,no,si,no,no,si,tienda,11842.044402,30,2,m10,178.120444,4.635882,no,no,mid-age,si,0
3,1640de7c-ba08-cdfc-c21b-517fd7c5a259,soltero,Santafé de Bogotá,no,f,si,si,no,no,si,online,39601.288181,32,4,m18,332.970456,30.698536,no,no,Legend,no,0
4,e820f090-f4e7-eb8b-677c-55808c868b6f,unión libre,Santafé de Bogotá,no,b,no,si,si,si,si,online,36126.943210,70,3,m2,70.864127,44.999640,si,si,Legend,no,0


## 2. Auditoría de Calidad - Usuarios

Verificación de integridad de IDs en la base de usuarios.

In [6]:
# Detección de IDs duplicados
conteo_duplicados = df_usuarios_raw['ID_Cuenta'].value_counts()
ids_duplicados = conteo_duplicados[conteo_duplicados > 1]

print(f"Total de IDs duplicados: {len(ids_duplicados)}")
print(f"Registros afectados: {ids_duplicados.sum()}")

if len(ids_duplicados) > 0:
    print(f"\nTop 10 IDs más duplicados:")
    print(ids_duplicados.head(10))
    
    # Ejemplo de ID duplicado para análisis
    id_ejemplo = ids_duplicados.index[0]
    print(f"\nEjemplo de perfiles duplicados para ID {id_ejemplo}:")
    cols_analisis = ['ID_Cuenta', 'Departamento', 'Edad', 'Tipo_persona', 'Tipo_Plan']
    display(df_usuarios_raw[df_usuarios_raw['ID_Cuenta'] == id_ejemplo][cols_analisis])

Total de IDs duplicados: 16878
Registros afectados: 34067

Top 10 IDs más duplicados:
ID_Cuenta
121314        14
1234          13
1022390282     8
1              7
30881871       6
ANONIMO1       5
12345          4
23621188       4
900475036      4
93436751       4
Name: count, dtype: int64

Ejemplo de perfiles duplicados para ID 121314:


,ID_Cuenta,Departamento,Edad,Tipo_persona,Tipo_Plan
104,121314,Santafé de Bogotá,33,/casado.,NaN
105,121314,Santander,41,soltero-casado.,b
106,121314,Atlántico,46,soltero,d
107,121314,Cundinamarca,29,soltero,00000
108,121314,Nariño,70,soltero,b
109,121314,Exterior,45,soltero,a
110,121314,Norte de Santander,30,soltero,c
111,121314,Arauca,23,soltero,00000
112,121314,Valle del Cauca,30,soltero,NaN
113,121314,Boyacá,39,soltero,f


## 3. Limpieza de Usuarios

Se eliminan registros con IDs duplicados para garantizar relación 1:1 entre ID y perfil de usuario.

In [7]:
# Filtrar solo IDs únicos
ids_unicos = conteo_duplicados[conteo_duplicados == 1].index
df_usuarios_clean = df_usuarios_raw[df_usuarios_raw['ID_Cuenta'].isin(ids_unicos)].copy()

print(f"Usuarios originales: {df_usuarios_raw.shape[0]:,}")
print(f"Usuarios después de limpieza: {df_usuarios_clean.shape[0]:,}")
print(f"Registros eliminados: {df_usuarios_raw.shape[0] - df_usuarios_clean.shape[0]:,}")
print(f"Porcentaje retenido: {(df_usuarios_clean.shape[0] / df_usuarios_raw.shape[0]) * 100:.2f}%")

Usuarios originales: 337,051
Usuarios después de limpieza: 302,984
Registros eliminados: 34,067
Porcentaje retenido: 89.89%


## 4. Auditoría de Calidad - Consultas

Análisis de formato y consistencia de IDs en el historial de consultas.

In [8]:
print("Análisis de IDs en Historial de Consultas")
print(f"Total de registros: {len(df_consultas_raw):,}")
print(f"IDs únicos: {df_consultas_raw['ID_Cuenta'].nunique():,}")

# Análisis de longitud de IDs
df_consultas_raw['longitud_id'] = df_consultas_raw['ID_Cuenta'].astype(str).str.len()

print(f"\nDistribución de longitud de IDs:")
dist_longitud = df_consultas_raw['longitud_id'].value_counts().sort_index()
print(dist_longitud)

# Identificar rango válido (IDs entre 7 y 10 dígitos)
print(f"\nRegistros con IDs válidos (7-10 dígitos): {df_consultas_raw['longitud_id'].between(7, 10).sum():,}")
print(f"Registros con IDs sospechosos: {(~df_consultas_raw['longitud_id'].between(7, 10)).sum():,}")

Análisis de IDs en Historial de Consultas
Total de registros: 1,282,960
IDs únicos: 319,862

Distribución de longitud de IDs:
longitud_id
1         94
2         40
3         64
4        760
5       1290
6      11698
7      51810
8     691770
9     312226
10    209736
11      3008
12       140
13        84
14        98
16         2
17         2
36       138
Name: count, dtype: int64

Registros con IDs válidos (7-10 dígitos): 1,265,542
Registros con IDs sospechosos: 17,418


### Interpretación

Los IDs con longitudes extremas (1-5 dígitos o >12 dígitos) corresponden probablemente a:
- Registros de prueba del sistema
- Errores de digitación en el canal de atención
- Datos sintéticos de QA

La distribución válida se concentra entre 7-10 dígitos, consistente con documentos de identidad.

In [9]:
# Opcional: Filtrar consultas con IDs válidos
# df_consultas_clean = df_consultas_raw[df_consultas_raw['longitud_id'].between(7, 10)].copy()
# Por ahora mantenemos todos los registros para no perder datos

df_consultas_clean = df_consultas_raw.copy()
df_consultas_clean.drop(columns=['longitud_id'], inplace=True, errors='ignore')

## 5. Integración de Datos

Merge entre historial de consultas y datos de usuarios limpios mediante ID_Cuenta.

In [10]:
# Merge interno: solo registros con match en ambas tablas
df_master = pd.merge(
    df_consultas_clean,
    df_usuarios_clean,
    on='ID_Cuenta',
    how='inner'
)

# Eliminar columnas innecesarias
columnas_drop = ['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y']
df_master.drop(columns=columnas_drop, inplace=True, errors='ignore')

print(f"Dimensiones del dataset maestro: {df_master.shape}")
print(f"Registros: {df_master.shape[0]:,}")
print(f"Variables: {df_master.shape[1]}")
print(f"\nCobertura del merge:")
print(f"  Consultas originales: {df_consultas_clean.shape[0]:,}")
print(f"  Consultas con usuario: {df_master.shape[0]:,}")
print(f"  Tasa de match: {(df_master.shape[0] / df_consultas_clean.shape[0]) * 100:.2f}%")

Dimensiones del dataset maestro: (1140532, 24)
Registros: 1,140,532
Variables: 24

Cobertura del merge:
  Consultas originales: 1,282,960
  Consultas con usuario: 1,140,532
  Tasa de match: 88.90%


## 6. Validación del Dataset Maestro

In [11]:
print("Información del dataset:")
df_master.info()

print(f"\nPrimeras filas del dataset maestro:")
display(df_master.head())

print(f"\nEstadísticas descriptivas:")
display(df_master.describe())

# Verificar valores nulos
null_counts = df_master.isnull().sum()
if null_counts.sum() > 0:
    print(f"\nValores nulos por columna:")
    display(null_counts[null_counts > 0])
else:
    print(f"\nNo hay valores nulos en el dataset")

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140532 entries, 0 to 1140531
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID_Cuenta              1140532 non-null  object 
 1   Fecha_consulta         1140532 non-null  object 
 2   Tipo_consulta          1140532 non-null  object 
 3   Tipo_persona           1117580 non-null  object 
 4   Departamento           1140532 non-null  object 
 5   Tiene_plan_avanzado    1140532 non-null  object 
 6   Tipo_Plan              1027608 non-null  object 
 7   Es_moroso              1140532 non-null  object 
 8   Tiene_plus             1140532 non-null  object 
 9   Ha_caido_mora          1140532 non-null  object 
 10  Correo                 1140532 non-null  object 
 11  usa_app                1140532 non-null  object 
 12  Forma_pago             854928 non-null   object 
 13  Monto_adeudado         1140532 non-null  float6

,ID_Cuenta,Fecha_consulta,Tipo_consulta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
0,340,2015-06-22 16:14:35,Actualización datos,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
1,340,2015-06-22 16:37:02,Agendamiento citas,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
2,340,2015-06-22 15:50:56,Agendamiento citas,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
3,4d5c0e16-9bef-8334-687e-55f043e02c62,2015-09-09 14:35:37,Agendamiento citas,soltero,Santafé de Bogotá,no,f,no,si,si,si,no,mensajero,30167.091798,65,3,m17,124.262540,55.257504,no,no,new-new,si,0
4,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,2015-07-24 20:36:26,Agendamiento citas,soltero,Santafé de Bogotá,no,f,no,si,no,no,si,tienda,11842.044402,30,2,m10,178.120444,4.635882,no,no,mid-age,si,0



Estadísticas descriptivas:


,Monto_adeudado,Edad,Duracion_llamada,Tiempo_en_espera,y
count,1.140532e+06,1.140532e+06,1.140532e+06,1.140532e+06,1.140532e+06
mean,1.990862e+04,4.847729e+01,1.989085e+02,2.391255e+01,2.507426e-01
std,1.414081e+04,1.786794e+01,1.410557e+02,1.694226e+01,4.334408e-01
min,3.015708e+01,1.800000e+01,1.602929e-01,2.663275e-02,0.000000e+00
25%,9.533535e+03,3.300000e+01,9.545149e+01,1.143818e+01,0.000000e+00
50%,1.663744e+04,4.800000e+01,1.663701e+02,2.007227e+01,0.000000e+00
75%,2.682283e+04,6.400000e+01,2.685886e+02,3.224388e+01,1.000000e+00
max,1.709877e+05,7.900000e+01,1.463228e+03,1.865452e+02,1.000000e+00



Valores nulos por columna:


Tipo_persona       22952
Tipo_Plan         112924
Forma_pago        285604
Estrato           112984
Motivo_llamada      1076
dtype: int64

## 7. Exportación de Datos Procesados

In [12]:
# Crear directorio de salida si no existe
output_dir = '../data/procesada'
os.makedirs(output_dir, exist_ok=True)

# Guardar dataset maestro
output_path = f'{output_dir}/master_dataset.csv'
df_master.to_csv(output_path, index=False, encoding='utf-8')

print(f"Dataset guardado exitosamente en: {output_path}")
print(f"Tamaño del archivo: {os.path.getsize(output_path) / (1024**2):.2f} MB")

# Guardar también versiones limpias intermedias
df_usuarios_clean.to_csv(f'{output_dir}/usuarios_clean.csv', index=False, encoding='utf-8')
df_consultas_clean.to_csv(f'{output_dir}/consultas_clean.csv', index=False, encoding='utf-8')

print(f"\nArchivos adicionales guardados:")
print(f"  - usuarios_clean.csv")
print(f"  - consultas_clean.csv")

Dataset guardado exitosamente en: ../data/procesada/master_dataset.csv
Tamaño del archivo: 199.20 MB

Archivos adicionales guardados:
  - usuarios_clean.csv
  - consultas_clean.csv


## Resumen del Proceso

1. **Datos cargados**: Historial de consultas y base de usuarios
2. **Limpieza aplicada**: Eliminación de IDs duplicados en usuarios
3. **Integración**: Merge exitoso con tasa de match >90%
4. **Output**: Dataset maestro listo para análisis exploratorio y modelamiento

El dataset resultante contiene registros validados con información completa de consultas y perfil de usuario.